In [2]:
import requests                 #Used to import file directly from the web
import io                       #Used to import file directly from the web
import random                   #Used to generate a random number
from datetime import datetime   #Used to determine Now()
from datetime import timedelta  #Used to sum timedeltas
import pandas as pd             #Used to work with data frames
import textwrap as tw           #Used to wrap text

# Question # 2: Perform the experiment to compare the performance of 2D list over Linear List by executing only Linear Search.
# (3 points)
# 1. Read the given text file and store the string data in Linear list straightforward but distributing
# the data in rows based on the first letter (lexicographical order).


#Import the file directly from the web
url = 'https://raw.githubusercontent.com/joelvinas/COMP-SCI_5501/aa4391ae9d3f2cb73e89f3b1fd5bc86bfe34fe2b/Assignment2/Data/words_alpha.txt'
response = requests.get(url)

response.raise_for_status()
if response.status_code == 200:
  wordList = response.text.splitlines()
else:
  print(f"Error: {response.status_code}")
  exit()

wordList.sort()
dictionary = {}
for word in wordList:
   firstLetter = word[0]
   if firstLetter not in dictionary:
     dictionary[firstLetter] = []
   dictionary[firstLetter].append(word)

# 2. Generate the keys randomly and search from the Linear and 2D list
# execute this process ten times. (Treatments)

#Generate Random Word list
randWordList = []
TotalOperations = 10
for randWord in range(TotalOperations):
  random_int = random.randint(1, len(wordList))
  randWordList.append(wordList[random_int])

class Treatment:
  iteration = 0
  random_word = ""
  starttime = ""
  endtime = ""
  def __init__(self, iteration, random_word, starttime):
    self.iteration = iteration
    self.random_word = random_word
    self.starttime = starttime
    self.comparisons = 0

  def duration(self):
    return self.endtime - self.starttime

def linear_search(index, wordArray, random_word):
  starttime = datetime.now()
  comparisons = 1
  wordIsFound = False
  curTreatment = Treatment(index, random_word, starttime)
  for curWord in wordArray:
    if curWord == random_word:
      curTreatment.endtime = datetime.now()
      wordIsFound = True
      break
    else:
      comparisons += 1
  if not wordIsFound:
    curTreatment.endtime = datetime.now()
  curTreatment.comparisons = comparisons
  return curTreatment

def getDataframe(method_name, results):
  column_names = ["method", "iteration", "random_word", "starttime", "endtime", "duration", "comparisons"]
  df = pd.DataFrame(columns = column_names)

  for curIteration in results:
    new_row = [method_name, curIteration.iteration, curIteration.random_word, curIteration.starttime, curIteration.endtime, curIteration.duration(), curIteration.comparisons]
    df.loc[len(df)] = new_row
  return df

#Linear Search (from Word List)
index = 0
linear_results_1D = []
for curWord in randWordList:
  index += 1
  starttime = datetime.now()
  curTreatment = linear_search(index, wordList, curWord)
  linear_results_1D.append(curTreatment)

#2D list search (from Dictionary)
# For each word, determine first letter
# using that, perform a Linear search within the array
index = 0
linear_results_2D = []
for curWord in randWordList:
  index += 1
  starttime = datetime.now()
  firstLetter = curWord[0]
  useList = dictionary[firstLetter]
  curTreatment = linear_search(index, useList, curWord) # Treatment(index, curWord, starttime)
  linear_results_2D.append(curTreatment)

# # 3. Compare the number of comparisons made in each treatment and give the results in tabulated form and also calculate the average.
df_all_results = pd.DataFrame()
df_all_results = pd.concat([df_all_results, getDataframe("Linear Search", linear_results_1D)], ignore_index=True)
df_all_results = pd.concat([df_all_results, getDataframe("2D Search", linear_results_2D)], ignore_index=True)
df_all_results['duration_µs'] = df_all_results['duration'].apply(lambda x: int((x.total_seconds() - int(x.total_seconds())) * 1000 * 1000))

df_summary = df_all_results.groupby(['method']).agg({'comparisons': 'mean', 'duration_µs': 'mean'}).reset_index() # ['duration_µs'].mean().reset_index()
display(df_summary)

# # 4. Exaplain your observations.
observations = "On average, it is clear that the 2D search method results in fewer comparisons and a shorter search duration. The heavy lifting was done during the initialization of the 2D array, where the larger dataset was segmented into up to 26 buckets. \nThis results in a smaller search area, similar to binary/ternary search. With a smaller search area, each search was faster with fewer comparisons.\nA similar process called 'Full-Text Indexing' is often used to optimize word-based searching of large blobs of text by sorting words found within a text, and indexing their placement.\nExpanding this methodology beyond the first letter and placing nested arrays by letter may be an improvement over the ternary search. The difficulty of search is then only as hard as the number of letters in the search word. For large enough word libraries, this method might be an improvement on the binary/ternary search."
wrapped_text = tw.fill(observations, width=70)
print(f"Observations:\n{wrapped_text}")

,method,comparisons,duration_µs
0,2D Search,12652.0,645.5
1,Linear Search,192238.2,10182.4


Observations:
On average, it is clear that the 2D search method results in fewer
comparisons and a shorter search duration. The heavy lifting was done
during the initialization of the 2D array, where the larger dataset
was segmented into up to 26 buckets.  This results in a smaller search
area, similar to binary/ternary search. With a smaller search area,
each search was faster with fewer comparisons. A similar process
called 'Full-Text Indexing' is often used to optimize word-based
searching of large blobs of text by sorting words found within a text,
and indexing their placement. Expanding this methodology beyond the
first letter and placing nested arrays by letter may be an improvement
over the ternary search. The difficulty of search is then only as hard
as the number of letters in the search word. For large enough word
libraries, this method might be an improvement on the binary/ternary
search.
